In [1]:
import numpy as np
import torch
import os
import sys
sys.path.append(os.path.join(os.path.split(os.getcwd())[0],'data_loader'))
# from data import DataBuildClassifier
from data_proj_pursuit_v2 import DataProjPursuit_v2 as DataBuildClassifier
from src.model_torch import EEGNet_experimental
from scipy.signal import welch
from scipy.fftpack import fft,rfft, fftshift, fftfreq, rfftfreq
import mne
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
params = {'resample_to': 369,
                 'D': 3,
                 'F1': 12,
                 'dropoutRate1': 0.52,
                 'dropoutRate2': 0.36,
                 'lr': 0.00066,
                 }

In [3]:
model = torch.load('./res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model')

In [9]:
# data = DataBuildClassifier('/home/likan_blk/BCI/NewData')
data = DataBuildClassifier('./with ica')
# all_subjects = [25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38]
# all_subjects = [6, 7, 8, 9]
all_subjects = 'subj 6'
# subjects = data.get_event_data(all_subjects, event='cl1', shuffle=False, window=[(0.2, 0.5)], baseline_window=(0.2, 0.3),
#                              resample_to=params['resample_to'])
subjects = data.get_event_data(all_subjects, shuffle=False, window=[-1.2, 1.2], baseline_window=(-0.1, 0),
                             event='cl1')
print(subjects)
print(subjects[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [10]:
x,y = subjects[25][0], subjects[25][1]
x = x.transpose(0,2,1)[:,np.newaxis,:,:]

ValueError: axes don't match array

In [ ]:
kernLength1 = params.get('kernLength1',64)
kernLength2 = params.get('kernLength2',16)
F2 = params.get('F2',params['F1']*params['D'])
model = EEGNet_experimental(nb_classes=2, Chans=subjects[25][0].shape[2], Samples = subjects[25][0].shape[1],
                               dropoutRates=(params['dropoutRate1'], params['dropoutRate1']),
                               kernLength1=kernLength1, kernLength2=kernLength2, poolKern1=4,
                               poolKern2=8,
                               F1=params['F1'],
                               D=params['D'], F2=F2)
# model.load_state_dict(torch.load('/home/likan_blk/BCI/eegnet_torch/res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model'))
model.load_state_dict(torch.load('./res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model'))

In [ ]:
spatial_filters = model.depthwise1.weight.cpu().detach().numpy()

In [ ]:
def read_info(path_to_file):
    with open(path_to_file,'r') as f:
        content = f.readlines()
    content = content[1:]
    print(len(content))
    res = np.zeros((len(content),2))
    for i in range(len(content)):
        splt_line = content[i].split()
        res[i,0] = np.pi*float(splt_line[1])/180 + np.pi/2
        res[i,1] = float(splt_line[2])
    return res
info = read_info('/home/likan_blk/BCI/NewData/order&locations.info')

In [ ]:
plt.polar(info[:,0],info[:,1],'ro')

In [ ]:
def pol2cart(phi,r ):
    x = r * np.cos(phi)
    y = r * np.sin(phi)
    return np.vstack((x, y)).T
cort_info = pol2cart(info[:,0], info[:,1])

plt.scatter(cort_info[:,0],cort_info[:,1])

In [ ]:
spatial_filters.shape

In [ ]:
mne.viz.plot_topomap(spatial_filters[0,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[1,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[2,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[3,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[4,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[5,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[6,0,:,0],cort_info)

In [ ]:
first_output = model.conv1(torch.Tensor(torch.Tensor(x)))
first_output.shape

In [ ]:
weights = model.conv1.weight.detach().cpu().numpy()[:,0,0,:]

In [ ]:
weights.shape

In [ ]:
yf = fft(weights[4,:])
xf = fftfreq(weights.shape[1], 1/params['resample_to'])
xf = fftshift(xf)
yplot = fftshift(yf)
plt.plot(xf, 1.0/weights.shape[1] * np.abs(yplot))

In [ ]:
fft(weights[4,:])

In [ ]:
rfft(weights[4,:])